# IN4050 - Genetic Algorithm
## Representations

### 1. ![Naming_Question](EA_Terms.png)

Name the terms shown in the picture above.

### Solution:
- **A:** Locus - the position of the gene - aka the index
- **B:** Allele - Possible values for the given gene
- **C:** Gene - one element in the array
- **D:** Genotype - a set of gene values
- **E:** Phenotype - what could be built based on the genotype

### 2. Mention some of the most common representations of genomes.

### Solution: 

- We want to make it as simple as possible for representing a genotypes. 
- Some ways to represent a genotype is: 
  - Array with 0s and 1s. The length of the array would be equal to the amount of genes. 0s represent the absence of a gene, while 1s represent the opposite.
  - Instead of array, use sequence of binary numbers. Same concept and uses less memory.
  - Instead of 1s and 0s, use sequence/array of characters. represents more information for each gene, but uses more memory. 

### 3. Perform a mutation operation on the representations given below.

binary = $[1, 0, 1, 1]$;
integer = $[4, 2, 4, 1]$;
real_valued = $[2.53, 1.42, 3.14, 1.68]$;
permutation = $[3, 4, 1, 2]$

In [1]:
import random
import numpy as np

# Arrays 
binary = [1, 0, 1, 1]
integer = [4, 2, 4, 1]
real_valued = [2.53, 1.42, 3.14, 1.68]
permutation = [3, 4, 1, 2]

# Method for printing the mutation
def print_mutation(original, mutated):
    print(f"Original: {original}, Mutated: {mutated}\n")

# Binary mutation
# 1. Select a random index
# 2. "Flop" the binary value to mutate it 
random_index = random.sample(range(len(binary)), 1)[0]
binary_cpy = list(binary)
if binary_cpy[random_index] == 0:
    binary_cpy[random_index] = 1
else:
    binary_cpy[random_index] = 0

print_mutation(binary, binary_cpy)


# Real valued or float mutation 
# 1. Select a random index
# 2. Set the value of the item at the index, from a uniform distribution from the array
#   - For real values, a discrete distribution  
#   - For floating points, a simple uniform distribution 

random_index = random.sample(range(len(integer)), 1)[0]
integer_cpy = list(integer)
integer_cpy[random_index] = np.random.randint(1,4)
print_mutation(integer, integer_cpy)


random_index = random.sample(range(len(real_valued)), 1)[0]
real_valued_cpy = list(real_valued)
high = max(real_valued_cpy)
low = min(real_valued_cpy)
real_valued_cpy[random_index] = round(np.random.uniform(low, high), 2)
print_mutation(real_valued, real_valued_cpy)


# For a permutation: 
# - Swap two values

i, j = random.sample(range(len(real_valued)), 2)
permutation_cpy = list(permutation)
temp = permutation_cpy[i]
permutation_cpy[i] = permutation_cpy[j]
permutation_cpy[j] = temp
print_mutation(permutation, permutation_cpy)


Original: [1, 0, 1, 1], Mutated: [0, 0, 1, 1]

Original: [4, 2, 4, 1], Mutated: [3, 2, 4, 1]

Original: [2.53, 1.42, 3.14, 1.68], Mutated: [2.53, 1.42, 3.14, 3.13]

Original: [3, 4, 1, 2], Mutated: [2, 4, 1, 3]



### 4. Given the sequences (2,4,7,1,3,6,8,9,5) and (5,9,8,6,2,4,1,3,7). Implement these algorithms to create a new pair of solutions: 

#### a. Partially mapped crossover (PMX)

In [2]:
# Add your solution here
def pmx_pair(a, b):
    assert(len(a) == 9)
    assert(len(b) == 9)

    # Create children from the given parents 
    c1 = list(a)
    c2 = list(b)

    # Replace the middle part with the middle part of the other parent
    c1[2:6] = b[2:6]
    c2[2:6] = a[2:6]

    # Fix conflicts for c1 (using a -> b mapping for swapped segment)
    for i in range(2, 6):
        if b[i] in c1[0:2] + c1[6:]:
            # Find the mapped value from the swapped section
            swap_val = b[i]
            while swap_val in c1[2:6]:
                swap_val = a[b.index(swap_val)]
            # Replace the duplicate value
            c1[c1.index(b[i])] = swap_val

    # Fix conflicts for c2 (using b -> a mapping for swapped segment)
    for i in range(2, 6):
        if a[i] in c2[0:2] + c2[6:]:
            # Find the mapped value from the swapped section
            swap_val = a[i]
            while swap_val in c2[2:6]:
                swap_val = b[a.index(swap_val)]
            # Replace the duplicate value
            c2[c2.index(a[i])] = swap_val


    return c1, c2


#### b. Order crossover

In [3]:
# Add your solution here
def order_crossover_pair(parent1, parent2):
    size = len(parent1)
    # Choose two crossover points
    start, end = sorted(random.sample(range(size), 2))
    
    # Initialize offspring
    offspring1 = [None] * size
    offspring2 = [None] * size

    # Copy the subsequences from parents to offspring
    offspring1[start:end+1] = parent1[start:end+1]
    offspring2[start:end+1] = parent2[start:end+1]

    def fill_order(offspring, parent):
        current_pos = (end + 1) % size
        parent_idx = (end + 1) % size
        while None in offspring:
            if parent[parent_idx] not in offspring:
                offspring[current_pos] = parent[parent_idx]
                current_pos = (current_pos + 1) % size
            parent_idx = (parent_idx + 1) % size

    # Fill the rest using order from the other parent
    fill_order(offspring1, parent2)
    fill_order(offspring2, parent1)

    return offspring1, offspring2

#### c. Cycle crossover

In [4]:
# Add your solution here
def cycle_crossover_pair(parent1, parent2):
    size = len(parent1)
    offspring1, offspring2 = [None] * size, [None] * size

    def cycle_map(parent1, parent2):
        index = 0
        visited_indices = set()
        cycle_indices = []
        while index not in visited_indices:
            visited_indices.add(index)
            cycle_indices.append(index)
            index = parent1.index(parent2[index])
        return cycle_indices

    # Crossover 1: Follow the cycles from parent1 to parent2
    cycle_indices = cycle_map(parent1, parent2)
    for i in range(size):
        if i in cycle_indices:
            offspring1[i] = parent1[i]
            offspring2[i] = parent2[i]
        else:
            offspring1[i] = parent2[i]
            offspring2[i] = parent1[i]

    return offspring1, offspring2

#### Test crossovers

In [5]:
a = [2, 4, 7, 1, 3, 6, 8, 9, 5]
b = [5, 9, 8, 6, 2, 4, 1, 3, 7]

c, d = pmx_pair(a, b)
e, f = order_crossover_pair(a, b)
g, h = cycle_crossover_pair(a, b)
print(f"Parents: {a} and {b}")
print(f"Children after PMX: {c} and {d}")
print(f"Children after Order Crossover: {e} and {f}")
print(f"Children after Cycle Crossover: {g} and {h}")

Parents: [2, 4, 7, 1, 3, 6, 8, 9, 5] and [5, 9, 8, 6, 2, 4, 1, 3, 7]
Children after PMX: [3, 1, 7, 6, 2, 4, 8, 9, 5] and [5, 9, 8, 4, 2, 6, 1, 3, 7]
Children after Order Crossover: [2, 4, 7, 1, 3, 5, 9, 8, 6] and [3, 9, 8, 6, 2, 5, 4, 7, 1]
Children after Cycle Crossover: [2, 4, 7, 1, 3, 6, 8, 9, 5] and [5, 9, 8, 6, 2, 4, 1, 3, 7]
